In [37]:
from astropy.io import fits
from astropy.stats import sigma_clip
from astropy.stats import biweight_midvariance as bimv
from astropy.stats import biweight_location as bil
from astropy.table import Table
from astropy.utils.data import get_pkg_data_filename
from collections import Counter
import glob
from matplotlib import cm
from matplotlib import pyplot as plt
% matplotlib inline
import numpy as np
import numpy.ma as ma
import operator
import scipy as sp
from scipy import signal
from scipy.interpolate import UnivariateSpline
from scipy.interpolate import splprep, splev
import sys
import time
import timeit

In [25]:
def correction(file, correction, sigma=2, poly=6):
    
    '''This version creates a global variable that is a matrix of the final output values. These
    can be called using the indicies of the IFU, and are given in chronological order'''
    
    '''This section creates the correction matricies that will be used later. Each matrix needs
    to hold the correction for the same column on the science frames, and we only want to apply
    the correction from the same IFU as the science frame. So, we create an array for both the
    left- and right-hand IFUs, and in this array each spot corresponds to a row and column on
    the CCD field, i.e. 045. This will allow us to pull the correct correction matrix for the
    science frame, by matching the row and column, as well as side.'''
    
    w, h = 10, 10;
    lcorrection_array = [[0 for x in range(w)] for y in range(h)]
    rcorrection_array = [[0 for x in range(w)] for y in range(h)]
    correction_list = dict()
    correction_data = dict()
    
    for alpha in correction:
        hdul = fits.open(alpha)
        correction_list[alpha] = [str(hdul[0].header['IFUSLOT']),str(hdul[0].header['CCDPOS'])]
        correction_data[alpha] = [hdul[0].data]
        row = int(correction_list[alpha][0][-1])
        column = int(correction_list[alpha][0][:-1])-1
        side = correction_list[alpha][1]

        correction_matrix = [[0 for x in range(correction_data[alpha][0].shape[1])] for y in range(correction_data[alpha][0].shape[0])]
        corr = correction_data[alpha][0].T
    
        for beta in range(np.shape(corr)[0]):
            ave = bil(corr[beta])
            fix = corr[beta]/ave

            for gamma in range(np.shape(corr)[1]):
                if fix[gamma] < 0.01:
                    fix[gamma] = 0
                correction_matrix[gamma][beta] = fix[gamma]

        if side=='L':
            lcorrection_array[row][column] = correction_matrix
        if side=='R':
            rcorrection_array[row][column] = correction_matrix
            
    '''This section makes the list of times of observations of the files being used.'''
    
    global times
    times = set([])
    file_list = dict()
    data_list = dict()   
    
    for delta in file:
        hdul = fits.open(delta)
        file_list[delta] = [str(hdul[0].header['IFUSLOT']),str(hdul[0].header['CCDPOS']),str(hdul[0].header['UT'])]
        data_list[delta] = [hdul[0].data]
        times.add(file_list[delta][2])
        
    times = sorted(times)
            
    global output_matrix
    output_matrix = [[[0 for z in range(len(times))] for x in range(10)] for y in range(10)]
    
    '''This section actually calculates output.'''
    
    '''We start by creating blank matrices.'''
    
    for epsilon in times:    
        w, h = 10, 10;
        lmatrix = [[0 for x in range(w)] for y in range(h)]
        rmatrix = [[0 for x in range(w)] for y in range(h)]
        matrix = [[0 for x in range(w)] for y in range(h)]
        
        timestamp = list(enumerate(times))[times.index(epsilon)][0]
        
        for zeta in file_list:
            row = int(file_list[zeta][0][-1])
            column = int(file_list[zeta][0][:-1])-1
            side = file_list[zeta][1]
            time = file_list[zeta][2]
            
            '''We only continue if the time matches. Each file will be done, but to correlate
            the correct ones together, they are sorted by time.'''
            
            if time==epsilon:
                
                '''Here is where we pull the correction matrix, which are stored by side, row, and
                column, as explained in the first step of the program.'''
                
                if side=='L':
                    correction_matrix = lcorrection_array[row][column]
                if side=='R':
                    correction_matrix = rcorrection_array[row][column]
                
                im = data_list[zeta][0]
                im = im/correction_matrix
                sc = sigma_clip(im)
                medfilt = sp.signal.medfilt(sc)
                medfilt = ma.masked_invalid(medfilt)
                rows = ma.mean(medfilt,axis=1)
                y = list(range(len(rows)))
                medpoly = np.poly1d(np.polyfit(y,rows,poly))
                flattened = rows - medpoly(y)
                source = []
                count = 0
                
                '''The data is flattened, in order to correctly indentify points that are errors.
                Normal clipping would not take into account the natural differences in various
                parts of the frame, and so might cut actual data and leave real errors. The
                degree fit can be changed when running the code.'''

                for eta in range(len(rows)):
                    mean = bil(flattened)         
                    sig = ma.std(flattened)
                    los = ma.where(flattened[eta]>(mean+sigma*sig))[0]
                    if los.size:
                        source.append(eta)

                rows_list = np.ndarray.tolist(rows)
                
                '''The ones marked as errors are removed from the data set.'''

                for theta in source:
                    del rows_list[(theta-count)]
                    count = count+1

                '''The average of the remaining rows is found, and added to the correct side
                matrix.'''    
                
                out = bil(rows_list)
                if out < 16:
                    out = 0
                if side=='L':
                    lmatrix[row][column] = out
                if side=='R':
                    rmatrix[row][column] = out

        '''The side matrices are averaged, for a final matrix. This is done for each time that
        data was taken, as calculated by the list of times section.'''            
                    
        for iota in range(10):
            for kappa in range(10):
                matrix[iota][kappa] = np.round(np.mean([lmatrix[iota][kappa],rmatrix[iota][kappa]]))
                #matrix[iota][kappa] = np.mean([lmatrix[iota][kappa],rmatrix[iota][kappa]])
                output_matrix[iota][kappa][timestamp] = np.mean([lmatrix[iota][kappa],rmatrix[iota][kappa]])
        
        print("Time:",epsilon)
        print(np.matrix(matrix))
        print("")

In [26]:
files=glob.glob("Fep*.fits")
corrections = glob.glob('../../cal/20180219_011102/Femastertwi*.fits')

In [27]:
correction(files,corrections)

/usr/lib/python3/dist-packages/numpy/lib/function_base.py:3558: RuntimeWarning: Invalid value encountered in median
  RuntimeWarning)
/home/rfoster/.local/lib/python3.5/site-packages/astropy/stats/biweight.py:110: RuntimeWarning: invalid value encountered in greater_equal
  mask = (np.abs(u) >= 1)
/home/rfoster/.local/lib/python3.5/site-packages/ipykernel_launcher.py:94: RuntimeWarning: divide by zero encountered in true_divide
/home/rfoster/.local/lib/python3.5/site-packages/ipykernel_launcher.py:94: RuntimeWarning: invalid value encountered in true_divide
/home/rfoster/.local/lib/python3.5/site-packages/astropy/stats/sigma_clipping.py:165: RuntimeWarning: invalid value encountered in greater
  _filtered_data.mask |= _filtered_data > max_value
/home/rfoster/.local/lib/python3.5/site-packages/astropy/stats/sigma_clipping.py:166: RuntimeWarning: invalid value encountered in less
  _filtered_data.mask |= _filtered_data < min_value


Time: 07:02:15.498
[[   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]
 [   0.    0.  241.    0.    0.    0.    0.    0.  219.  234.]
 [   0.    0.  229.    0.    0.    0.  250.  219.  220.  184.]
 [   0.    0.    0.  218.    0.    0.    0.  213.  150.  196.]
 [   0.    0.    0.  225.    0.    0.    0.  220.  190.  189.]
 [   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]]

Time: 07:13:18.880
[[   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]
 [   0.    0.  265.    0.    0.    0.    0.    0.  239.  257.]
 [   0.    0.  249.    0.    0.    0.  272.  238.  242.  201.]
 [   0.    0.  